# Introduction

L'apprentissage automatique (*machine learning*) peut sembler intimidant avec son jargon issu du monde de l'informatique et des statistiques. Pourtant si l'on commence par les bases et que l'on augmente la difficulté progressivement, il est tout à fait possible d'arriver à assimiler les concepts fondamentaux de ce domaine.

Ce cours vous donnera un aperçu de la manière dont les *data scientists* élaborent, conçoivent et mettent en application leurs modèles de ML. Vous pourrez alors vous servir de cette base pour continuer à vous perfectionner tout seul, ou bien vous arrêter là après avoir acquis les connaissances nécessaires pour comprendre et dialoguer avec les experts du monde de la data.

# Cas-pratique : l'immobilier

Le premier jeu de données que nous allons utiliser concerne l'immobilier. Dans la vie réelle les agents immobiliers savent estimer la valeur d'un bien car ils associent aux différentes caractéristiques d'un bien (nombre de pièces, surface, emplacement etc.) un prix grâce à leur expérience.

Le programme que nous allons réaliser va nous permettre de réaliser nous aussi des estimations, c'est-à-dire de de prédire une valeur donnée, mais c'est cette fois c'est l'ordinateur qui va "apprendre" de lui-même à grâce aux données que nous allons lui fournir.

## Arbre de décision

Nous utiliserons pour l'instant un modèle appelé "arbre de décision". Ce sont des modèles très instinctifs, faciles à comprendre et à analyser qui pourront rendre service dans bien des cas.

Commençons par un exemple très simple :

<div>
<img src="files/decision_tree_1.png" alt="CPU" width="50%" align='center'/> </div>

Ce modèle divise les maisons en deux catégories : celle ayant 2 chambres ou moins et celle ayant plus de 2 chambres, puis il affiche le prix moyen de chaque groupe.

Le modèle utilise le jeu de données pour décider comment répartir les maisons dans ces deux groupes, puis à nouveau pour prédire le prix dans chaque groupe. L'étape qui consiste à définir les paramètres d'un modèle à partir de données est appelée entraînement, *fitting* ou **training**. Les données utilisées pour paramétrer ce modèle sont appelées données d'entraînement (*training data*).

Les détails de la manière dont le modèle est entraîné (par exemple, la manière de diviser les données) sont assez complexes et nous ne les étudierons que plus tard. Une fois le modèle ajusté, nous pourrons l'appliquer à de nouvelles données afin de prédire le prix d'un logement.

Nous pouvons prendre en compte davantage de facteurs en utilisant un arbre qui a plus de "divisions" (*splits*), c'est-à-dire qui est "plus profond". Un arbre de décision qui prend également en compte la taille du terrain de chaque maison pourrait ressembler à ceci.

<div>
<img src="files/decision_tree_2.png" alt="CPU" width="60%" align='center'/> </div>

Pour prédire le prix d'une maison, on parcourt l'arbre de décision, en choisissant toujours le chemin correspondant aux caractéristiques de cette maison. Le prix prédit pour la maison se trouve en bas de l'arbre et le point en bas de l'arbre où nous faisons une prédiction s'appelle une feuille (*leaf*).

# Exploration avec Pandas

In [ ]:
import pandas as pd
df = pd.read_csv("data/iowa_housing.csv")

In [ ]:
df.shape

In [ ]:
df.columns

# Valeurs manquantes

In [ ]:
df.isna()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum().loc[df.isna().sum() > 0]

In [ ]:
max_col_len = len(max(df.columns, key=len))
max_val_len = len(str(max(df.isna().sum(), key=lambda x : len(str(x)))))

for i, num in zip(df.isna().sum().index, df.isna().sum()):
    print(f'{i}{(max_col_len - len(i)) * " "} | Missing values : {num}{(max_val_len - len(str(num))) * " "} | Completion : {round(100 - (num / df.shape[0] * 100))}%') 

# Statistics

In [ ]:
df['lotarea'].mean()

In [ ]:
df['lotarea'].mean().round()

In [ ]:
df['saleprice'].mean()

In [ ]:
df['saleprice'].mean().round()

In [ ]:
df.describe(include='all')

In [ ]:
df['yearbuilt'].max()

In [ ]:
df['yearbuilt'].min()

In [ ]:
df['yearbuilt'].describe()

# Variable cible

La variable cible aussi appelée variable à expliquer, variable dépendante, variable à prédire, ou encore *target* est la variable que nous voulons prédire. Elle est symbolisée par un "y".

Ici il s'agit de la dernière colonne de notre dataframe qui contient le prix de vente du bien immobilier : ``'saleprice'``

In [ ]:
y = df['saleprice']

# Variables explicatives

Les variables explicatives, aussi appellées variables à expliquer, variables prédictives ou encore `'features'` sont les variables d'entrées (*input*) de notre modèle. Ce sont grâce à elles que le modèle va pouvoir déterminer la valeur de notre variable de sortie (*output*). Elles sont symbolisées par un "X".

Le choix de ces variables a une grande incidence sur les résultats. Parfois nous utiliserons toutes les variables que nous avons à disposition, parfois nous n'utiliserons qu'une partie d'entre elles. Il existe un grand nombre de différentes méthodes (logiques, scientifiques, statistiques, informatiques etc.) pour nous aider à faire ce choix.

Ici nous utiliserons les variables suivantes comme *features* :

In [ ]:
feature_names = [
'lotarea',
'yearbuilt',
'1stflrsf',
'2ndflrsf',
'fullbath',
'bedroomabvgr',
'totrmsabvgrd',
]

In [ ]:
X = df[feature_names]

In [ ]:
X.describe()

In [ ]:
X.isna().sum()

# Modélisation

### Choix du modèle

Nous allons choisir un "arbre de décision" aussi appelé *DecisionTreeRegressor* que nous appellerons iowa_model.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# random_state will allow model reproducibility
iowa_model = DecisionTreeRegressor(random_state=42)

### Ajustement du modèle *fit*

L'entraînement du modèle est très simple : une seule ligne de code suffit ! Par convention, on donne d'abord les *features* puis la *target*.

In [ ]:
iowa_model.fit(X,y)

### Visualisation

Une fois notre modèle crée, on peut le visualiser à l'aide de différentes manières.

In [ ]:
from sklearn import tree
# print(tree.export_text(iowa_model))

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(25,20))
tree.plot_tree(iowa_model,
               feature_names=X.columns,
               max_depth=2,
               filled=True)

plt.show()

### Prédictions

Notre modèle peut désormais prédire des valeurs à partir d'un ensemble de variables. Essayons de lui faire prédire les résultats à partir des *features* (X).

In [ ]:
y_pred = iowa_model.predict(X)
print(y_pred)

Comparons les 5 premières prédictions avec les 5 premières valeurs de y.

In [ ]:
list(y_pred[:5])

In [ ]:
list(y[:5])

C'est étrange on s'attendrait à ce que notre modèle soit un peux faux, mais celui-ci arrive à prédire au dollar près notre variable y !
Vérifions cela avec un peu de Pandas.

In [ ]:
res = pd.DataFrame({'y':y,'y_pred':y_pred})
res['y_pred'] = res['y_pred'].astype(int)
res['diff'] = res['y'] - res['y_pred'].round()
res

In [ ]:
res.loc[res['diff'] != 0]

Sur les 2930 prédictions effectuées, seules 71 ne sont pas correctes. Et pourtant même celles-ci ne sont pas très éloignées du résultat attendus. Avons-nous créé le meilleur modèle possible ?

# Validation d'un modèle

Chaque modèle, une fois ajusté, doit être évalué à l'aide de différentes métriques. Une bonne métrique pour évaluer une valeur continue, comme c'est le cas ici, est d'examiner la précision de la prédiction. Pour chaque bien immobilier on va donc calculer la valeur absolue de la différence entre la vraie valeur et la valeur prédite par le modèle :

erreur = |vraie valeur - valeur prédite|

Si l'on fait ensuite la moyenne de ces valeurs, cela nous donne la MAE (*Mean Absolute Error*). La MAE nous indique quel est l'écart moyen d'une prédiction avec la valeur réelle.

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y, y_pred)

La précision est donc excellente, mais c'est parce que la manière dont nous avons ajusté notre modèle n'est pas la bonne. Comme les données avec lesquelles nous avons entraîné puis testé le modèle sont les mêmes, il est normal que nous n'obtenons quasiment que des bonnes réponses. Nous avons ici affaire à un cas classique *d'overfitting*.

Pour évaluer la robustesse d'un modèle nous allons le tester sur des données que le modèle n'a encore jamais vues. Pour cela nous allons diviser nos données en deux groupes : l'une d'elle va servir à l'apprentissage (*train*), et l'autre à tester le modèle (*test*).

Le paramètre "train_size" va déterminer la proportion de nos données qui va être utilisée pour l'entraînement. Une valeur de 0.8 signifie que l'on réserve 80% de nos données pour l'apprentissage et 20% pour le test.

In [ ]:
from sklearn.model_selection import train_test_split
# Now our data are split in 4 different parts
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

iowa_model.fit(X_train, y_train)

y_pred = iowa_model.predict(X_test)
print(mean_absolute_error(y_test, y_pred))

La MAE est considérablement plus élevée, environ 200 fois ! Comme le prix moyen d'une maison était de l'ordre de $180 000, cela veut dire que notre modèle se trompe d'environ 1/6 du prix. Il y a, bien évidemment, de nombreux moyens d'obtenir un score plus élevé.

# Les paramètres du modèle

Un arbre de décision peut être paramétré de nombreuses manières différentes, comme vous pouvez le constater en examinant la [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) relative à notre type de modèle.

### Différence entre paramètres et hyperparamètres

En ML, les hyperparamètres sont les paramètres qui vont encadrer le processus de génération des paramètres internes au modèle.

Par exemple dans notre modèle ses paramètres consistent, notamment, en toutes les ramifications qui aboutissent aux feuilles de notre arbre. Ces paramètres ont été déterminés par le modèle au cours de son apprentissage et ont beaucoup varié entre le moment où l'on a lancé l'ajustement et le moment où celui-ci s'est terminé.

Les hyperparamètres sont les paramètres qui sont le plus souvent donné par un être humain et qui ne seront pas modifiés au cours de l'apprentissage. Il s'agit de directions générales, de paramètres de plus haut niveau.

L'un des hyperparamètre les plus importants de ce modèle est la profondeur de l'arbre. Pour l'instant nous ne lui avions donné aucune directive, donc cet hyperparamètre a été généré par le programme. Examinons-le :

In [ ]:
iowa_model.tree_.max_depth

Il y a au maximum 26 niveau de profondeurs dans notre arbre. A chaque fois que nous ajoutons un niveau de profondeur à notre arbre nous augmentons son nombre maximal de feuilles et donc sa précision. Mais, en contrepartie, le nombre de maisons dans chaque feuille sera de plus en plus réduit ce qui veut dire que les prédictions seront de moins en moins fiables. Il va donc falloir trouver un compromis entre précision et fiabilité.

### *Overfitting* et *underfitting*

Les phénomènes d'*overfitting* et d'*underfitting* sont des concepts centraux du *machine learning*.

- On parle d'*overfitting* lorsque les résultats de notre modèle collent de très près aux données avec lesquelles il a été entraîné mais se trompe beaucoup lorsqu'on l'applique sur des données inconnues. C'est ce qui arrivera si notre arbre de décision est trop profond.

- On parle d'*underfitting* lorsque le modèle n'arrive pas à faire la distinction entre des caractéristiques essentielles de nos données. Celui-ci aura un mauvais score sur les données d'entraînement, mais aussi sur les données réservées pour le test. C'est ce qui arrivera si notre modèle n'est pas assez profond.

<div>
<img src="files/underfitting_and_overfitting.png" alt="CPU" width="75%" align='center'/> </div>

Le graphique ci-dessus montre la variation de la MAE en fonction de la profondeur d'un arbre de décision. Le terme de "validation" désigne ici le jeu de "test". Quelques remarques sur le graphique :

- Le modèle aura, en moyenne, toujours un meilleur score lorsqu'il prédit des données qui proviennent de son jeu d'entraînement plutôt que des données inconnues provenant du jeu de test.

- L'augmentation de la profondeur va permettre dans un premier temps d'améliorer le modèle que ce soit sur le *train* ou le *test*.

- Il arrive un moment où l'augmentation de la profondeur apportera un gain de précision sur le *train* mais ou la MAE va elle commencer à augmenter sur le *test*. C'est le phénomène d'overtiffing.

Le but des hyperparamètres est donc de trouver ce point d'équilibre qui devrait nous permettre de maximiser les résultats du modèle.

In [ ]:
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    return(mae)

In [ ]:
# Exemple
get_mae(50, X_train, X_test, y_train, y_test)

In [ ]:
d = {}
for max_leaf_nodes in [2, 5, 25, 30, 40, 50, 100, 200]:
    mae = get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test)
    d[max_leaf_nodes] = mae
    print(f"Max leaf nodes: {max_leaf_nodes} \t\t Mean Absolute Error: {mae}")

In [ ]:
(pd.DataFrame.from_dict(d.items())
             .rename(columns={0 : 'max_leaf_nodes', 1 : 'mae'})
             .plot(x='max_leaf_nodes', y='mae', color='red'));

Graphiquement le meilleur hyperparamètre semble être autour de 40. On pourrait le trouver automatiquement à l'aide de certaines techniques que nous verrons plus tard.

# Quel avenir pour notre modèle ?

Une fois avoir trouvé les meilleurs hyperparamètres, on peut entraîner à nouveau le modèle mais en utilisant cette fois le jeu de données en entier afin d'améliorer encore un peu la précision. Puis nous pourrions le tester sur des données réelles, provenant d'un jeu de données différents afin de voir comment il se comporte.

Une autre possibilité serait d'utiliser un modèle différent et voir si celui-ci se comporte mieux ou moins bien.

# Pour aller plus loin

### Sélection de colonnes sur la base du taux de valeurs manquantes

Un ``df.dropna()`` appliqué sur toutes les colonnes nous supprimerait toutes les lignes de notre dataframe. On peut cependant choisir d'éliminer toutes les colonnes qui ont un taux de complétion inférieures à 94% (par exemple), ce qui nous permet de garder au moins 94% de notre jeu de données.

In [ ]:
# Exemple 1
(df['garagetype'].isna().sum() / df.shape[0]) < 0.06

In [ ]:
# Exemple 2
(df['miscfeature'].isna().sum() / df.shape[0]) < 0.06

In [ ]:
cols_to_keep = [col for col in df.columns if (df[col].isna().sum() / df.shape[0]) < 0.06]

In [ ]:
df.shape, df.dropna().shape, df[good_cols].dropna().shape

In [ ]:
new_df = df[cols_to_keep].dropna()

In [ ]:
(new_df.isna().sum() > 0).sum()